# Python Intro - Final Exercise


In [1]:
import pyodbc
import pandas as pd
import numpy as np


In [2]:
### In windows, Using a ODBC DNS (predefined connection name)
import pyodbc
conn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=DESKTOP-N6AVJGE\SQLEXPRESS;"
                      "Database=COLLEGE;"
                      "Trusted_Connection=yes;")


In [3]:
### Get the students table
import pandas as pd
students = pd.read_sql("SELECT * FROM Students;",conn)
students.head()

,StudentId,FirstName,LastName,Gender
0,162.0,Elmira,Holston,F
1,154.0,Sierra,Van Vorst,F
2,28.0,Mycah,Bingham,F
3,54.0,Alia,Jones,F
4,31.0,Chanda,Fillmore,F


In [4]:
### Get the Courses table
courses = pd.read_sql("SELECT * FROM Courses;",conn)
courses.head()

,CourseId,CourseName,DepartmentID,TeacherId
0,1.0,English Begginers,1.0,15.0
1,2.0,Advanced English,1.0,15.0
2,3.0,Proffesional English,1.0,15.0
3,4.0,Mathematics 101,2.0,2.0
4,5.0,Matematics Advanced,2.0,13.0


In [5]:
### Get the Classrooms table
classrooms = pd.read_sql("SELECT * FROM Classrooms;",conn)
classrooms.head()

,CourseId,StudentId,degree
0,1.0,202.0,92.031070
1,1.0,12.0,81.175916
2,1.0,232.0,82.396587
3,1.0,123.0,62.420449
4,1.0,274.0,72.569900


In [6]:
### Get the Departments table
departments = pd.read_sql("SELECT * FROM Departments;",conn)
departments.head()

,DepartmentId,DepartmentName
0,1.0,English
1,2.0,Science
2,3.0,Arts
3,4.0,Sport


In [7]:
### Get the Teachers table
teachers = pd.read_sql("SELECT * FROM Teachers;",conn)
teachers.head()

,TeacherId,FirstName,LastName,Gender
0,1.0,Kaylea,Monzingo,F
1,2.0,Amanda,Dworkin,F
2,3.0,Jordyn,Baron,F
3,4.0,Katherine,Dibari,F
4,5.0,Heather,Metcalf,F


# Questions
 
 
## Q1. Count the number of students on each department

In [8]:
###inner merge courses and classrooms by CoureId
numstud=pd.merge(courses,classrooms,on='CourseId')

numstud.head(80)

,CourseId,CourseName,DepartmentID,TeacherId,StudentId,degree
0,1.0,English Begginers,1.0,15.0,202.0,92.031070
1,1.0,English Begginers,1.0,15.0,12.0,81.175916
2,1.0,English Begginers,1.0,15.0,232.0,82.396587
3,1.0,English Begginers,1.0,15.0,123.0,62.420449
4,1.0,English Begginers,1.0,15.0,274.0,72.569900
...,...,...,...,...,...,...
75,3.0,Proffesional English,1.0,15.0,26.0,82.740856
76,3.0,Proffesional English,1.0,15.0,146.0,84.785099
77,3.0,Proffesional English,1.0,15.0,28.0,85.842696
78,3.0,Proffesional English,1.0,15.0,109.0,81.613981


In [9]:
###count distinct the number of students, groupby deprtmentID. merge with deprtments to get the Department names. 
###since the column name is different, we need to assign the column name for each side.
pd.merge((numstud.groupby('DepartmentID').agg({'StudentId': pd.Series.nunique}))
         ,departments,left_on='DepartmentID',right_on='DepartmentId')



,StudentId,DepartmentId,DepartmentName
0,84.0,1.0,English
1,200.0,2.0,Science
2,137.0,3.0,Arts
3,81.0,4.0,Sport


In [13]:
###inner merge courses and classrooms on CourseId, merge with deprtments and 
### aggregation of the number of students, groupby deprtmentID, to get the Department names. 
###since the column name written differently, we need to refer to each column at each merging side.
#pd.merge((pd.merge(courses,classrooms,on='CourseId')        
#     .groupby('DepartmentID').agg({'StudentId': pd.Series.nunique}))
#         ,departments,left_on='DepartmentID',right_on='DepartmentId')

## Q2. How many students have each course of the English department and the total number of students in the department?

In [14]:

#dep_class_stud=pd.merge((numstud.groupby('DepartmentID').agg({'StudentId': 'nunique'}))
#         ,departments,left_on='DepartmentID',right_on='DepartmentId')
#dep_class_stud

In [18]:
#we aggregate the number of students, in all courses belong to deprtment Id==1(English). 
df=numstud[numstud.DepartmentID==1].groupby('CourseName').agg(Students=('StudentId','nunique'))
df

,Students
CourseName,
Advanced English,24
English Begginers,32
Proffesional English,38


In [19]:
#we sum the total number of students and added the name 'Total'
df.append(df.sum().rename('Total'))

,Students
CourseName,
Advanced English,24
English Begginers,32
Proffesional English,38
Total,94


## Q3. How many small (<22 students) and large (22+ students) classrooms are needed for the Science department?

In [27]:
#we count the number of students at each course that belong to the science department (departmentID==2)
df1=numstud[numstud.DepartmentID==2].groupby('CourseName').agg(Students=('StudentId','nunique'))
df1

,Students
CourseName,
Algebra,12
Biology,36
Chemistry,23
Computer Science,38
Data Science,30
Genetics,37
Geometry,13
Java,16
Matematics Advanced,10


In [28]:
#we defined big when the value at the students column >= 22 and small when it is less than 22 
big=df1.loc[df1['Students'] >= 22]
small=df1.loc[df1['Students'] < 22]

In [33]:
#we create a dataFrame with the names of columns and rows and the sum of big and small classes

data = {'classroom_size':['Big Classroom','Small classrooms'],'num_classrooms':[big.count()[0],small.count()[0]]} 
class_size=pd.DataFrame(data)
class_size

,classroom_size,num_classrooms
0,Big Classroom,10
1,Small classrooms,6


## Q4. A feminist student claims that there are more male than female in the College. Justify if the argument is correct

In [36]:
#we counted the number of students at each Gender
df=students.groupby('Gender').agg(Students=('StudentId','nunique'))
df

,Students
Gender,
F,165
M,115


## Q5. For which courses the percentage of male/female students is over 70%?

In [38]:
numstud1=pd.merge(numstud,students,on='StudentId')

numstud1.head(80)

,CourseId,CourseName,DepartmentID,TeacherId,StudentId,degree,FirstName,LastName,Gender
0,1.0,English Begginers,1.0,15.0,202.0,92.031070,Autumn,Major,F
1,4.0,Mathematics 101,2.0,2.0,202.0,73.387328,Autumn,Major,F
2,8.0,Chemistry,2.0,14.0,202.0,83.563932,Autumn,Major,F
3,18.0,R,2.0,17.0,202.0,66.109849,Autumn,Major,F
4,20.0,History of Arts,3.0,8.0,202.0,63.988269,Autumn,Major,F
...,...,...,...,...,...,...,...,...,...
75,10.0,Biology,2.0,3.0,42.0,90.002664,Melissa,Crespin,F
76,18.0,R,2.0,17.0,42.0,77.074106,Melissa,Crespin,F
77,24.0,Theater,3.0,21.0,42.0,80.953548,Melissa,Crespin,F
78,1.0,English Begginers,1.0,15.0,231.0,81.608274,Malena,Wofford,F


In [115]:
#aggregartion total number of students at each course
#aggregartion number of students at each course where Gender is Female
df=numstud1.groupby('CourseName').agg(total=('StudentId',pd.Series.nunique))
male=numstud1[numstud1.Gender=='M'].groupby('CourseName').agg(male=('StudentId',pd.Series.nunique))
female=numstud1[numstud1.Gender=='F'].groupby('CourseName').agg(female=('StudentId',pd.Series.nunique))
male

,male
CourseName,
Advanced English,11.0
Algebra,4.0
Biology,17.0
Chemistry,8.0
Computer Science,20.0
Data Science,18.0
English Begginers,12.0
Genetics,11.0
Geometry,5.0


In [133]:
#merging df1 with df.
#create a new column by Dividing the number of female(female)*100 to total number of students (total) to get the percentage
f_m=df.merge(male,on='CourseName').merge(female,on='CourseName')
f_m['Female_perc'] = f_m.female/ (f_m.male+f_m.female)*100
f_m['Male_perc'] = f_m.male/ (f_m.male+f_m.female)*100
f_m

,total,male,female,Female_perc,Male_perc
CourseName,,,,,
Advanced English,24.0,11.0,13.0,54.166667,45.833333
Algebra,12.0,4.0,8.0,66.666667,33.333333
Biology,36.0,17.0,19.0,52.777778,47.222222
Chemistry,23.0,8.0,15.0,65.217391,34.782609
Computer Science,38.0,20.0,18.0,47.368421,52.631579
Data Science,30.0,18.0,12.0,40.000000,60.000000
English Begginers,32.0,12.0,20.0,62.500000,37.500000
Genetics,37.0,11.0,26.0,70.270270,29.729730
Geometry,13.0,5.0,8.0,61.538462,38.461538


In [134]:
#subsetting courses which female and or male percentage is over 70% 
f_m[(f_m.Female_perc > 70.00) | (f_m.Male_perc > 70.00)]


,total,male,female,Female_perc,Male_perc
CourseName,,,,,
Genetics,37.0,11.0,26.0,70.270270,29.729730
Sculpture,23.0,6.0,17.0,73.913043,26.086957
Tenis,18.0,5.0,13.0,72.222222,27.777778


## Q6. For each department, how many students passed with a grades over 80?

In [137]:
#we counted the number of students with grade>80, grouped by departments 
pd.merge(numstud1[numstud1.degree> 80].groupby('DepartmentID').agg(PassedOver_80=('StudentId','nunique'))
          ,departments,left_on='DepartmentID',right_on='DepartmentId')

,PassedOver_80,DepartmentId,DepartmentName
0,40,1.0,English
1,122,2.0,Science
2,61,3.0,Arts
3,39,4.0,Sport


## Q7. For each department, how many students passed with a grades under 60?

In [138]:
##we counted the number of students with grade<60, grouped by departments 
pd.merge(numstud1[numstud1.degree< 60].groupby('DepartmentID').agg(PassedUnder_60=('StudentId','nunique'))
          ,departments,left_on='DepartmentID',right_on='DepartmentId')

,PassedUnder_60,DepartmentId,DepartmentName
0,2,1.0,English
1,7,2.0,Science
2,3,3.0,Arts
3,1,4.0,Sport


## Q8. Rate the teachers by their average student's grades (in descending order).

In [143]:
#getting the averages of students' degrees for each teacher. 
df=numstud1.groupby('TeacherId').agg(Avg_degree=('degree','mean'))
df

,Avg_degree
TeacherId,
1.0,79.365537
2.0,76.899881
3.0,80.729494
5.0,80.440242
6.0,77.743575
8.0,75.955453
9.0,77.406852
10.0,78.383348
11.0,76.765296


In [145]:
#merging the TeacherId with their First and Last names from 'teacher' table
df1=df.merge(teachers,on='TeacherId')
df1

,TeacherId,Avg_degree,FirstName,LastName,Gender
0,1.0,79.365537,Kaylea,Monzingo,F
1,2.0,76.899881,Amanda,Dworkin,F
2,3.0,80.729494,Jordyn,Baron,F
3,5.0,80.440242,Heather,Metcalf,F
4,6.0,77.743575,Madison,Batan,F
5,8.0,75.955453,Caroline,Knuesel,F
6,9.0,77.406852,Katerina,Young,F
7,10.0,78.383348,Mackenzie,Olsen,F
8,11.0,76.765296,Kaitlin,Delude,F
9,12.0,77.134323,Mikayla,Russell,F


In [148]:
#Picking the Teacher's name, ID and degree columns and sorting the table by the 'avg_degree' column
df1.loc[0:,['Avg_degree','LastName','FirstName','TeacherId']].sort_values(ascending=False,by='Avg_degree')



,Avg_degree,LastName,FirstName,TeacherId
19,81.432703,Hess,Darick,22.0
2,80.729494,Baron,Jordyn,3.0
3,80.440242,Metcalf,Heather,5.0
17,79.365867,Healy,Zachary,20.0
0,79.365537,Monzingo,Kaylea,1.0
15,78.908281,Bland,Daniel,18.0
21,78.814631,Price,Keegan,24.0
12,78.699492,Mckittrick,Conor,15.0
20,78.673509,Romkee,Maxwell,23.0
16,78.495479,Beaty,Terran,19.0


## Q9. Create a dataframe showing the courses, departments they are associated with, the teacher in each course, and the number of students enrolled in the course (for each course, department and teacher show the names).

In [149]:
#merging numstud1 table with 'teacher' table
numstud2=pd.merge(numstud1,teachers,on='TeacherId')

numstud2.head(20)

,CourseId,CourseName,DepartmentID,TeacherId,StudentId,degree,FirstName_x,LastName_x,Gender_x,FirstName_y,LastName_y,Gender_y
0,1.0,English Begginers,1.0,15.0,202.0,92.031070,Autumn,Major,F,Conor,Mckittrick,M
1,1.0,English Begginers,1.0,15.0,12.0,81.175916,Sophiya,Aponte,F,Conor,Mckittrick,M
2,2.0,Advanced English,1.0,15.0,12.0,85.115642,Sophiya,Aponte,F,Conor,Mckittrick,M
3,1.0,English Begginers,1.0,15.0,232.0,82.396587,Victoria,Kishinevsky,F,Conor,Mckittrick,M
4,1.0,English Begginers,1.0,15.0,123.0,62.420449,Kelsi,Taber,F,Conor,Mckittrick,M
5,1.0,English Begginers,1.0,15.0,274.0,72.569900,Samantha,Anderson,F,Conor,Mckittrick,M
6,1.0,English Begginers,1.0,15.0,150.0,79.511241,Emily,Beach,F,Conor,Mckittrick,M
7,1.0,English Begginers,1.0,15.0,161.0,65.438025,Cullen,Brewton,M,Conor,Mckittrick,M
8,1.0,English Begginers,1.0,15.0,39.0,100.000000,Kayli,May,F,Conor,Mckittrick,M
9,1.0,English Begginers,1.0,15.0,251.0,80.603352,Heather,Williams,F,Conor,Mckittrick,M


In [150]:
#counting the number of student for each course 
numstudclass=numstud2.groupby('CourseId').agg(num_students=('StudentId','nunique'))
numstudclass



,num_students
CourseId,
1.0,32
2.0,24
3.0,38
4.0,18
5.0,10
6.0,12
7.0,13
8.0,23
9.0,12


In [152]:
df=pd.merge(courses,departments,left_on='DepartmentID',right_on='DepartmentId')
df1=df.set_index('TeacherId').join(teachers.set_index('TeacherId'))
df2=pd.merge(df1,numstudclass,on='CourseId')#.sort_values(ascending=True,by='CourseId')
df2.loc[0:,['CourseId','CourseName','DepartmentName','FirstName','LastName','num_students']].sort_values(ascending=True,by='CourseId')


,CourseId,CourseName,DepartmentName,FirstName,LastName,num_students
15,1.0,English Begginers,English,Conor,Mckittrick,32
16,2.0,Advanced English,English,Conor,Mckittrick,24
17,3.0,Proffesional English,English,Conor,Mckittrick,38
1,4.0,Mathematics 101,Science,Amanda,Dworkin,18
10,5.0,Matematics Advanced,Science,Alexander,Watts,10
11,6.0,Algebra,Science,Alexander,Watts,12
12,7.0,Geometry,Science,Alexander,Watts,13
14,8.0,Chemistry,Science,Jeffrey,Batzel,23
13,9.0,Physics,Science,Alexander,Watts,12
2,10.0,Biology,Science,Jordyn,Baron,36


## Q10. Create a dataframe showing the students, the number of courses they take, the average of the grades per class, and their overall average (for each student show the student name).

In [166]:
numstud3=pd.merge(numstud2,departments,left_on='DepartmentID',right_on='DepartmentId')
numstud3
numstud4=numstud3.loc[0:,['StudentId','FirstName_x','LastName_x','degree','DepartmentName']].sort_values(ascending=True,by='StudentId')
numstud4


,StudentId,FirstName_x,LastName_x,degree,DepartmentName
402,1.0,Alexandria,Heller,86.419914,Science
253,1.0,Alexandria,Heller,79.381125,Science
308,1.0,Alexandria,Heller,72.871487,Science
582,1.0,Alexandria,Heller,86.677535,Arts
417,2.0,Cassie,Liska,81.156681,Science
...,...,...,...,...,...
529,280.0,Matthew,Mcgrath,82.738913,Arts
440,280.0,Matthew,Mcgrath,79.089894,Science
587,280.0,Matthew,Mcgrath,65.742483,Arts
281,280.0,Matthew,Mcgrath,72.303258,Science


In [173]:
#calculate the total grades' average (mean) and the total number of courses (nunique) for each students 
numclass=numstud4.groupby('StudentId').agg({'degree':['mean','nunique']})
numclass

degree        
                mean nunique
StudentId                   
1.0        81.337515       4
2.0        79.137364       5
3.0        78.793700       3
4.0        71.481003       3
5.0        76.718650       3
...              ...     ...
275.0      82.028830       4
276.0      76.784735       3
277.0      80.482335       6
278.0      75.359189       3
280.0      76.621429       5

[256 rows x 2 columns]

In [171]:
#calculate grades' average for each students grouped by departments
studentdegree=numstud4.groupby(['StudentId','FirstName_x','LastName_x','DepartmentName']).agg({'degree':'mean'})
studentdegree


degree
StudentId FirstName_x LastName_x DepartmentName           
1.0        Alexandria Heller     Arts            86.677535
                                 Science         79.557508
2.0        Cassie     Liska      Arts            79.542738
                                 Science         78.867115
3.0        Hayley     Frank      English         83.972245
...                                                    ...
277.0      Bryan      Medina     Science         81.298837
278.0      Sydney     Blas       Science         72.578017
                                 Sport           76.749775
280.0      Matthew    Mcgrath    Arts            77.237998
                                 Science         75.696576

[502 rows x 1 columns]

In [174]:
#unstanking the departmentName
studentdegree1=studentdegree.unstack(level=3)

s2 = studentdegree1.join(numclass, on='StudentId')
s2


degree                                    \
DepartmentName                         Arts    English     Science      Sport   
StudentId FirstName_x LastName_x                                                
1.0        Alexandria Heller      86.677535        NaN   79.557508        NaN   
2.0        Cassie     Liska       79.542738        NaN   78.867115        NaN   
3.0        Hayley     Frank             NaN  83.972245   76.204427        NaN   
4.0        Maxwell    Kotch       69.356144  70.929801   74.157066        NaN   
5.0        Dylan      Boehm       64.494845        NaN  100.000000  65.661106   
...                                     ...        ...         ...        ...   
275.0      Elise      Ohler       94.702880  71.617451   80.897496        NaN   
276.0      Dylan      Bannister         NaN        NaN   76.784735        NaN   
277.0      Bryan      Medina      78.849330        NaN   81.298837        NaN   
278.0      Sydney     Blas              NaN        NaN   72.578017  76.749775   
280.0      Matthew    Mcgrath     77.237998        NaN   75.696576        NaN   

                                                     
DepartmentName                         mean nunique  
StudentId FirstName_x LastName_x                     
1.0        Alexandria Heller      81.337515       4  
2.0        Cassie     Liska       79.137364       5  
3.0        Hayley     Frank       78.793700       3  
4.0        Maxwell    Kotch       71.481003       3  
5.0        Dylan      Boehm       76.718650       3  
...                                     ...     ...  
275.0      Elise      Ohler       82.028830       4  
276.0      Dylan      Bannister   76.784735       3  
277.0      Bryan      Medina      80.482335       6  
278.0      Sydney     Blas        75.359189       3  
280.0      Matthew    Mcgrath     76.621429       5  

[256 rows x 6 columns]